In [10]:
#tf 활용 bmi 판정
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv("./dataset/bmi.csv")

#키와 몸무게 정규화
df["height"] = df["height"] / 200
df["weight"] = df["weight"] / 100

#label 컬럼 변환 - thin[1, 0, 0]/normal[0, 1, 0]/fat [0, 0, 1]
bclass = {"thin": [1, 0, 0] , "normal":[0, 1, 0], "fat": [0, 0, 1]}
df["label_pat"] = df["label"].apply(lambda x: np.array(bclass[x]))

#학습데이터와 테스트 데이터 분류
test_df = df[15000:20000]
test_pat= test_df[["weight", "height"]]
test_ans = list(test_df["label_pat"])

X = tf.placeholder(tf.float32, [None, 2]) #키, 몸부게 데이터 담을 placeholder  선언
Y = tf.placeholder(tf.float32, [None, 3])   #정답 레이블 데이터 담을 placeholder  선언

W = tf.Variable(tf.zeros([2, 3])) 
b = tf.Variable(tf.zeros([3])) 

y = tf.nn.softmax(tf.matmul(X, W) + b)  #소프트맥스 회귀 정의
cross_entropy = -tf.reduce_sum(Y * tf.log(y))  #오차함수 - 교차 엔트로피
train= tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)  #경사하강법으로 학습
 
#예측값, 정답률 계산
predict = tf.equal(tf.argmax(y, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer()) 
for step in range(3501):
    i = (step*100) % 14000
    rows = df[i+1:i+1+100]
    x_pat = rows[["weight", "height"]]
    y_ans =  list(rows["label_pat"])
    sess.run(train, feed_dict={X: x_pat  , Y: y_ans })
    
    if step%500  == 0 :
        cre = sess.run(cross_entropy, feed_dict={X: x_pat  , Y: y_ans })
        acc = sess.run(accuracy , feed_dict={X: test_pat  , Y: test_ans })
        print("Epoch=", step, "오차=", cre, "정확률(평균)=", acc)

Epoch= 0 오차= 108.66269 정확률(평균)= 0.3242
Epoch= 500 오차= 57.58866 정확률(평균)= 0.8904
Epoch= 1000 오차= 45.020916 정확률(평균)= 0.898
Epoch= 1500 오차= 41.654335 정확률(평균)= 0.9566
Epoch= 2000 오차= 34.664024 정확률(평균)= 0.943
Epoch= 2500 오차= 34.287025 정확률(평균)= 0.9674
Epoch= 3000 오차= 26.880762 정확률(평균)= 0.9726
Epoch= 3500 오차= 29.590666 정확률(평균)= 0.9728


In [26]:
#동일코드 tf > keras 변환
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping


df = pd.read_csv("./dataset/bmi.csv")

#키와 몸무게 정규화
df["height"] /= 200
df["weight"] /= 100

X = df[["weight", "height"]].as_matrix()

#label 컬럼 변환 - thin[1, 0, 0]/normal[0, 1, 0]/fat [0, 0, 1]
bclass = {"thin": [1, 0, 0] , "normal":[0, 1, 0], "fat": [0, 0, 1]}
Y = np.empty((20000, 3))
for i, v in enumerate(df["label"]):
                      Y[i] = bclass[v]
 
#학습데인터 , 테스트 데이터 분리
X_train, Y_train = X[1:15001], Y[1:15001]
X_test, Y_test = X[15001:20001], Y[15001:20001]

model = Sequential()  #모델 객체 생성
model.add(Dense(512, input_shape=(2, ))) #Dense(노드수, ...) 층을 의미하는 객체
model.add(Activation('relu')) #활성화함수
model.add(Dropout(0.1))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Dense(3)) #분류 항목 수
model.add(Activation('softmax')) #활성화함수

model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])
hist = model.fit(X_train, Y_train, batch_size=100, epochs=20, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=2)], verbose=1)
#모델훈련                    
score = model.evaluate(X_test, Y_test) #모델평가(evaluate)
print("loss=", score[0])
print("accuracy=", score[1])

#weight decay(가중치 감소) - 학습중 가중치가 큰 것에 대해서 패널티를 부과해 과적합의 위험을 줄이는 방법
#Dropout - 복잡한 신경망에서 가중치 감소만으로 과적합을 피하기 어려운 경우 뉴런의 연결을 임의로 삭제시켜 신호를 전달하지 못하도록 하는 방법
#softmax 회귀 - 입력받은 값을 출력으로 0~1사이의 값으로 모두 정규화하여 출력값들의 총합은 항상 1이 되는 특성의 함수 분류하고 싶은 클래스 수 만큼 출력으로 구성

C:\Users\student\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


Train on 13500 samples, validate on 1500 samples
Epoch 1/20
13500/13500 [==============================] - 1s 59us/step - loss: 0.5227 - accuracy: 0.7847 - val_loss: 0.3248 - val_accuracy: 0.8573
Epoch 2/20
13500/13500 [==============================] - 1s 47us/step - loss: 0.2587 - accuracy: 0.8969 - val_loss: 0.2532 - val_accuracy: 0.8720
Epoch 3/20
13500/13500 [==============================] - 1s 46us/step - loss: 0.1975 - accuracy: 0.9164 - val_loss: 0.1385 - val_accuracy: 0.9487
Epoch 4/20
13500/13500 [==============================] - 1s 46us/step - loss: 0.1679 - accuracy: 0.9301 - val_loss: 0.3613 - val_accuracy: 0.8340
Epoch 5/20
13500/13500 [==============================] - 1s 46us/step - loss: 0.1552 - accuracy: 0.9298 - val_loss: 0.0849 - val_accuracy: 0.9900
Epoch 6/20
13500/13500 [==============================] - 1s 46us/step - loss: 0.1469 - accuracy: 0.9357 - val_loss: 0.1202 - val_accuracy: 0.9400
Epoch 7/20
13500/13500 [==============================] - 1s 46us/ste

In [28]:
from keras.models import Sequential
from keras.layers import Dense

import numpy as np
import tensorflow as tf

seed = 0 #실행마다 같은 결과를 출력하기 위해 설정
np.random.seed(seed)
tf.set_random_seed(seed)

Data_set = np.loadtxt("./dataset/ThoraricSurgery.csv",delimiter=",")

#환자 - 수술결과 구분
X = Data_set[:,0:17]
Y = Data_set[:,17]
#딥러닝 구조겨정
model = Sequential()
#첫번째 은닉층에 input_dim 통해 첫번째 dense가 은닉층 +입력층 겸함
model.add(Dense(30, input_dim=17, activation='relu')) #데이터에서 17개의 값 수취 은닉층의 30개 노드로 전송
#activation = 출력층 전달시 사용 활성화 함수
model.add(Dense(1, activation='sigmoid'))
#출력층 노드 갯수 1, 최종출력값에 사용되는 활성화 함수

#딥러닝 실행 (오차함수=RMSE)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(X, Y, epochs=30, batch_size=10)

print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))

Epoch 1/30
470/470 [==============================] - 0s 193us/step - loss: 0.6613 - accuracy: 0.3149
Epoch 2/30
470/470 [==============================] - 0s 67us/step - loss: 0.1488 - accuracy: 0.8511
Epoch 3/30
470/470 [==============================] - 0s 59us/step - loss: 0.1488 - accuracy: 0.8511
Epoch 4/30
470/470 [==============================] - 0s 65us/step - loss: 0.1488 - accuracy: 0.8511
Epoch 5/30
470/470 [==============================] - 0s 64us/step - loss: 0.1488 - accuracy: 0.8511
Epoch 6/30
470/470 [==============================] - 0s 65us/step - loss: 0.1487 - accuracy: 0.8511
Epoch 7/30
470/470 [==============================] - 0s 63us/step - loss: 0.1487 - accuracy: 0.8511
Epoch 8/30
470/470 [==============================] - 0s 64us/step - loss: 0.1487 - accuracy: 0.8511
Epoch 9/30
470/470 [==============================] - 0s 61us/step - loss: 0.1487 - accuracy: 0.8511
Epoch 10/30
470/470 [==============================] - 0s 60us/step - loss: 0.1486 - accur